### Import packages:

In [1]:
# import dva_sdhdf_combine
import imp
import os
import subprocess
import h5py
import numpy as np
from astropy.time import Time
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import datetime
import matplotlib.dates as mdates
from matplotlib.dates import HourLocator as HourLocator
from matplotlib.dates import MinuteLocator as MinuteLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy import units as u
from astropy.time import TimeDelta
from ipywidgets import interact

#### Change the directory to where the files are located" ####
day ='25'
directory = '../DVA/Data_Files/DVA_Day_Surveys/'

### Change the scan details to your current scan
df = 0.5        
dt = 6.9444431574083865e-06

#TODO: actually do the scan properties definition in the beginning
##############################################################

### Read in the file listing azimuth scan start and stop times:

In [2]:
scan_id = []    # The scan id number
scan_start = []  # Start time of the scan (UTC)
scan_stop = []   # Stop time of the scan (UTC)

# Read in the data and store it in arrays:
with open(directory+'DVAsurvey_phase1_day0'+day+'.txt') as fp:
    for line in fp:       
        scan_id.append(int(line.split()[0]))
        scan_start.append(line.split()[1]+'T'+line.split()[2][0:12])
        scan_stop.append(line.split()[3]+'T'+line.split()[4][0:12])
        
# Print out the scan numbers with their start and stop times:
for i in range(0,len(scan_id)):
    print(f"{scan_id[i]:04}",scan_start[i],scan_stop[i])

# Convert start and stop times to Modified Julian Day (MJD).
# This is needed for plotting and for selecting out data collected
# between particular times:
scan_start_mjd = Time(scan_start, format='isot',scale='utc').mjd
scan_stop_mjd  = Time(scan_stop,  format='isot',scale='utc').mjd


1052 2022-07-04T03:57:53.110 2022-07-04T04:15:55.184
2565 2022-07-04T04:20:37.406 2022-07-04T04:38:39.310
2566 2022-07-04T04:40:22.266 2022-07-04T04:58:24.003
1271 2022-07-04T05:03:42.716 2022-07-04T05:21:44.818
1272 2022-07-04T05:23:27.291 2022-07-04T05:41:29.292
2857 2022-07-04T05:48:35.269 2022-07-04T06:06:37.317
0124 2022-07-04T06:11:19.190 2022-07-04T06:29:21.101
1637 2022-07-04T06:34:03.920 2022-07-04T06:52:06.005
0342 2022-07-04T06:57:23.897 2022-07-04T07:15:25.930
1927 2022-07-04T07:21:55.804 2022-07-04T07:39:58.034
1928 2022-07-04T07:42:16.530 2022-07-04T08:00:18.319
2001 2022-07-04T08:03:48.814 2022-07-04T08:21:51.062
2002 2022-07-04T08:24:09.373 2022-07-04T08:42:11.545
2075 2022-07-04T08:45:41.821 2022-07-04T09:03:44.188
2148 2022-07-04T09:07:50.625 2022-07-04T09:25:52.832
2149 2022-07-04T09:27:35.229 2022-07-04T09:45:37.044
2150 2022-07-04T09:47:56.026 2022-07-04T10:05:57.828
2151 2022-07-04T10:07:40.752 2022-07-04T10:25:42.662
2152 2022-07-04T10:27:25.410 2022-07-04T10:45:

## Read in scan files and stich them together:

In [15]:
t_set = []
az_set = []
dec_set = []
ra_set = []
el_set = []
noise_set = []
trim_flag = []

scan0 = f"{scan_id[0]:04}"

# Use one of the scans to get the list of frequencies:
file = h5py.File(directory+'dva_survey_raw_scan_'+scan0+'.h5','r')
freq = file['data']['beam_0']['band_SB0']['frequency'][::12]/1e6

# Create empty arrays for the power data:
RR_set = np.empty([0,len(freq)])
LL_set = np.empty([0,len(freq)])
reRL_set = np.empty([0,len(freq)])
imRL_set = np.empty([0,len(freq)])

# Loop through all the scans in the "scan_num" list:
for i in scan_id[0:5]:
    print(i)
    # select the file:
    file = h5py.File(directory+'dva_survey_raw_scan_'+f"{i:04}"+'.h5','r')
    print(file)
    
    # access the correct location in the file structure:
    dataset = file['data']['beam_0']['band_SB0']['scan_0']
    
    # Add the position and time data to the corresponding arrays:
    dec_set = np.concatenate([dec_set,dataset['metadata']['declination']])
    ra_set = np.concatenate([ra_set,dataset['metadata']['right_ascension']])
    el_set = np.concatenate([el_set,dataset['metadata']['elevation']])
    az_set = np.concatenate([az_set,dataset['metadata']['azimuth']])
    t_set = np.concatenate([t_set,dataset['metadata']['utc']])
    noise_set = np.concatenate([noise_set,dataset['metadata']['noise_state']]) #This is a "mask" for noise regions 1 = noise 0=all good
    trim_flag = np.concatenate([trim_flag,dataset['metadata']['trim_scan_flag']])
    
    # Add the spectrometer power data to the corresponding arrays:
    freq_channel_increment = 12 #TODO: I'll have to change this to 1 once I am sure I'm reading the data correctly
    RR_set = np.concatenate([RR_set,dataset['data'][:,0,::freq_channel_increment]],axis=0)
    LL_set = np.concatenate([LL_set,dataset['data'][:,1,::freq_channel_increment]],axis=0)
    reRL_set = np.concatenate([reRL_set,dataset['data'][:,2,::freq_channel_increment]],axis=0)
    imRL_set = np.concatenate([imRL_set,dataset['data'][:,3,::freq_channel_increment]],axis=0)
    
t_plt = Time(t_set, format='isot',scale='utc').mjd

1052
<HDF5 file "dva_survey_raw_scan_1052.h5" (mode r)>
2565
<HDF5 file "dva_survey_raw_scan_2565.h5" (mode r)>
2566
<HDF5 file "dva_survey_raw_scan_2566.h5" (mode r)>
1271
<HDF5 file "dva_survey_raw_scan_1271.h5" (mode r)>
1272
<HDF5 file "dva_survey_raw_scan_1272.h5" (mode r)>


### Polarized Intensity

In [8]:
polarized = []
for i,j in zip(reRL_set,imRL_set):
    PI = np.sqrt((i**2)+(j**2))
    polarized.append(PI)
polarized_plot = np.array(polarized)

In [31]:
print(LL_set)
print("LL_set:", np.shape(LL_set))
print(" t_plt:", np.shape(t_plt))
print("  freq:", np.shape(freq))

[[ 9427295.5  9604977.   9833076.  ...  4204203.5  4204848.5  4116815. ]
 [ 9372518.5  9628859.   9894430.5 ...  4213138.5  4268423.5  4169953.5]
 [ 9504722.5  9551835.   9922202.5 ...  4202703.   4153782.5  4136170.5]
 ...
 [ 9665815.   9943170.5 10399451.  ...  4200309.   4201457.5  4162063. ]
 [ 9685502.5  9977550.  10460294.5 ...  4222053.5  4204084.   4170644.5]
 [ 9665752.5  9823968.  10389098.5 ...  4202247.   4210489.5  4185444.5]]
LL_set: (9018, 688)
 t_plt: (9018,)
  freq: (688,)


In [47]:
df = freq[1]-freq[0]

def DVA_Waterfall_View():
    #TODO: Add another DVA_Waterfall interactive_function such that I can change between LL_Set, RR_set, etc...
    power_min = 70
    power_max = 78

    fig,axs = plt.subplots(1,1,figsize=(15,10)) 
    fs = 16

    im = axs.imshow(10.*np.log10(LL_set.T),aspect='auto',vmin=power_min,vmax=power_max,origin='lower',
                extent=[t_plt[0],t_plt[-1],freq[0],freq[-1]])

    divider = make_axes_locatable(axs)
    cax = divider.append_axes('right', size='2%', pad=0.05)
    cbar = fig.colorbar(im, cax=cax, orientation='vertical')
    cbar.ax.tick_params(labelsize=fs) 
    cbar.set_label('Power (dB)', fontsize=fs)

    axs.set_xlim(t_plt[0],t_plt[-1])
    axs.set_ylim(freq[0],freq[-1])
    axs.tick_params(axis='both', labelsize=fs)
    axs.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    axs.fmt_xdata = mdates.DateFormatter('%H:%M:%S')
    axs.set_xlabel('Time (UTC)',fontsize=fs)        
    axs.set_ylabel('Frequency (MHz)',fontsize=fs)


def DVA_Cross_Sections(freq_chosen, time_idx, freq_cross_section):
    freq_measured = np.where(abs(freq-freq_chosen)<df)[0][0]

    fs = 12    
    fig,axs1 = plt.subplots(1,1,figsize=(16,6))  
    # TODO: use the log of LL_set_clean
    if(freq_cross_section):
        axs1.plot(freq, LL_set[time_idx,:], label = 'LL_set with polarized mask')
        axs1.vlines(freq_chosen, 0 , 100e9, color = 'red')
        axs1.set_ylim(np.min(LL_set[:,freq_measured]), np.max(LL_set[time_idx,:]))
        axs1.set_xlabel('Frequency',fontsize=fs)
        axs1.set_ylabel('Power',fontsize=fs)
    else:
        axs1.plot(t_plt, LL_set[:,freq_measured], label = 'LL_set unmasked RFI')
        axs1.vlines(t_plt[time_idx], 0 , 100e9, color = 'red')

        axs1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
        axs1.set_ylim(np.min(LL_set[:,freq_measured]), np.max(LL_set[:,freq_measured]))
        axs1.fmt_xdata = mdates.DateFormatter('%H:%M:%S')
        axs1.set_xlabel('Time (UTC)',fontsize=fs)
        axs1.set_ylabel('Power',fontsize=fs)
        axs1.legend()

def DVA_Visualization(waterfall_enabled):
    if waterfall_enabled:
        interact(DVA_Waterfall_View)
    else:
        interact(DVA_Cross_Sections, freq_chosen = (350, 1000, df), freq_cross_section = False, time_idx = (0,len(t_plt)-1))


interact(DVA_Visualization, waterfall_enabled = True)

#TODO: plot power in DB

interactive(children=(Checkbox(value=True, description='waterfall_enabled'), Output()), _dom_classes=('widget-…

<function __main__.DVA_Visualization(waterfall_enabled)>